In [52]:
import os

In [53]:
%pwd

'C:\\Users\\vinay\\OneDrive\\Documents\\GitHub\\Forest-WildFire-Detection-DeepL-Azure'

In [54]:
os.chdir("C:/Users/vinay/OneDrive/Documents/GitHub/Forest-WildFire-Detection-DeepL-Azure")

In [55]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [56]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories    


In [57]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir 
            )

            return data_ingestion_config
      

In [58]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [59]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [63]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-01 20:29:35,950: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-01 20:29:35,953: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-01 20:29:35,955: INFO: common: created directory at: artifacts]
[2025-07-01 20:29:35,957: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-01 20:29:42,166: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26065842
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2c21ce75795cb81ac7aa02425c00b5c36c78e28a509bbf9c6780b4a5a33b3aa4"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 35EC:1072CD:1A3ED9:226A3E:68647D70
Accept-Ranges: bytes
Date: Wed, 02 Jul 2025 00:29:38 GMT
Via: 1.1 varnish
X-Served-By: cache-yul1970020-YUL
X-Cache